In [ ]:
from pointcloud_invariance_smoothing.data import get_dataset
from pointcloud_invariance_smoothing.models.utils import get_model
import torch
import numpy as np
from torch.nn.functional import cross_entropy
from tqdm import tqdm

In [ ]:
dataset = {
    'name': 'modelnet40',
    'data_folder': '/your/path/to/modelnet/modelnet40_normal_resampled',
    'val_percentage': 0.2
}

model_config = {
    'model_type': 'pointnet',
    'model_params': {
        'input_tnet': True,
        'feature_tnet': False,
        'n_classes': 40,
        'n_point_dim': 3,
        'n_feat_dim': 0
    },
    'invariance_wrapper_params': {
        'wrapper_type': 'no_wrapper',
        'random_sign': False,
        'random_order': False
    }
}

In [ ]:
_, _, data_test = get_dataset(**dataset)

In [ ]:
model = get_model(**model_config)

train_file = torch.load('./equivariance_modelnet40_train_1')
state_dict_file = train_file['state_dict_best_acc']

model.load_state_dict(state_dict_file)
model = model.cuda()
model.eval()

for param in model.parameters():
    param.requires_grad = False

In [ ]:
pred = []
targets = []

for X, target in data_test:

    if isinstance(X, np.ndarray):
        X = torch.Tensor(X)

    X = X.unsqueeze(0).cuda()
    logits = model(X)[0]

    pred.append(int(logits.argmax().detach().cpu()))
    targets.append(int(target))

pred = np.array(pred)
targets = np.array(targets)

In [ ]:
(pred == targets).mean()

In [ ]:
results_dict = {
    'targets': targets,
    'pred': pred,
    'test_accuracy': (pred == targets).mean()
}

In [ ]:
def get_normalized_input_grad(X, target, model):
    X_perturbed = X.clone()
    X_perturbed.requires_grad = True

    logits = model(X_perturbed)[0]

    loss = cross_entropy(logits, torch.LongTensor([target]).cuda())
    loss.backward()

    input_grad = X_perturbed.grad.detach()
    input_grad_norm = torch.norm(input_grad, p=2, dim=[1, 2])
    assert input_grad_norm[0] > 0
    
    X_perturbed.requires_grad = False
    normalized_grad = (input_grad / input_grad_norm).detach()

    return normalized_grad

In [ ]:
budget_max = 1
budget_steps = 100

budgets = np.linspace(0, budget_max, budget_steps)
pred_at_budget = np.zeros((budget_steps, len(data_test)))


for i, (X, target) in enumerate(tqdm(data_test)):

    if isinstance(X, np.ndarray):
        X = torch.Tensor(X)

    X = X.unsqueeze(0).cuda()

    normalized_grad = get_normalized_input_grad(X, target, model)

    with torch.no_grad():

        for j, budget in enumerate(budgets):
            X_perturbed = X + budget * normalized_grad
            logits = model(X_perturbed)[0]

            pred_at_budget[j, i] = int(logits.argmax().detach().cpu())

In [ ]:
attacked = (pred_at_budget != pred)

In [ ]:
results_dict.update({
    'attacked': attacked,
})

In [ ]:
torch.save(results_dict,
           './results')